- The above data is data for prebid auctions. There are bidders who receive requests for placing ads, they respond to these requests with a bid and the highest bid wins the auction. Analyse the dataset to answer which bidder has the highest win rate (wins / total_bid). 

In [68]:
import pandas as pd 
pd.set_option('display.max_columns',None)
import os

In [69]:
df = pd.read_csv(os.path.join('analysis_id(in).csv'))

In [70]:
df.head()

,Unnamed: 0,date_hour,client,device_type,time_zone,os_name,browser_name,ad_unit,size,bidder,bid_range,media_type,request_count,response_count,prebid_win_count,win_count,sum_bid,sum_time_to_respond,median,min_bid,max_bid,avg_bid,sum_2nd_highest_bid,sum_prebid_winning_bid,sum_winning_bid,sum_nobid,sum_timeout
0,0,2021-04-11T01:00:00,asc126828,Desktop,America/Chicago,Linux,Chrome-85.04183133,div-gpt-ad-BT-D-Footers,728x90,onemobile,0.5-0.6,display,4,4,1.0,NaN,2.34080,6614.0,NaN,0.58520,0.58520,0.585200,0.5852,0.5852,NaN,NaN,NaN
1,1,2021-04-11T01:00:00,asc126828,Desktop,America/Chicago,Linux,Chrome-85.04183133,div-gpt-ad-BT-D-Footers,728x90,rhythmone,0.3-0.4,display,3,3,NaN,NaN,1.03230,5171.0,NaN,0.33945,0.34875,0.344100,NaN,NaN,NaN,NaN,NaN
2,2,2021-04-11T01:00:00,asc126828,Desktop,America/Chicago,Linux,Chrome-85.04183133,div-gpt-ad-BT-D-InLine-1,300x250,undertone,0.9-1,display,1,1,NaN,NaN,0.96000,1523.0,NaN,0.96000,0.96000,0.960000,NaN,NaN,NaN,NaN,NaN
3,3,2021-04-11T01:00:00,asc126828,Desktop,America/Chicago,Linux,Chrome-85.04183133,div-gpt-ad-BT-D-O-300x250-1,300x250,,,display,43,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2021-04-11T01:00:00,asc126828,Desktop,America/Chicago,Linux,Chrome-85.04183133,div-gpt-ad-BT-D-O-300x250-1,300x250,rhythmone,0.3-0.4,display,2,2,NaN,NaN,0.69285,3570.0,0.34875,0.34410,0.34875,0.346425,NaN,NaN,NaN,NaN,NaN


In [86]:
df.isnull().sum()

Unnamed: 0                   0
date_hour                    0
client                       0
device_type                  0
time_zone                    0
os_name                      0
browser_name                 0
ad_unit                      0
size                         0
bidder                       0
bid_range                    0
media_type                   0
request_count                0
response_count               0
prebid_win_count          3833
win_count                 4814
sum_bid                   1210
sum_time_to_respond       1210
median                    3240
min_bid                   1210
max_bid                   1210
avg_bid                   1210
sum_2nd_highest_bid       4266
sum_prebid_winning_bid    3833
sum_winning_bid           4814
sum_nobid                 5000
sum_timeout               5000
target                       0
dtype: int64

In [72]:
df.columns

Index(['Unnamed: 0', 'date_hour', 'client', 'device_type', 'time_zone',
       'os_name', 'browser_name', 'ad_unit', 'size', 'bidder', 'bid_range',
       'media_type', 'request_count', 'response_count', 'prebid_win_count',
       'win_count', 'sum_bid', 'sum_time_to_respond', 'median', 'min_bid',
       'max_bid', 'avg_bid', 'sum_2nd_highest_bid', 'sum_prebid_winning_bid',
       'sum_winning_bid', 'sum_nobid', 'sum_timeout'],
      dtype='object')

In [73]:
df['bidder'].value_counts(normalize=True)

bidder
                0.2420
ix              0.0928
onemobile       0.0814
medianet        0.0768
criteo          0.0612
pubmatic        0.0596
rhythmone       0.0492
33across        0.0480
triplelift      0.0464
openx           0.0394
appnexus        0.0394
rubicon         0.0356
emx_digital     0.0338
sharethrough    0.0278
pulsepoint      0.0268
sovrn           0.0204
undertone       0.0098
teads           0.0096
Name: proportion, dtype: float64

In [74]:
df[['bidder','request_count','response_count','prebid_win_count']].head()

,bidder,request_count,response_count,prebid_win_count
0,onemobile,4,4,1.0
1,rhythmone,3,3,NaN
2,undertone,1,1,NaN
3,,43,0,NaN
4,rhythmone,2,2,NaN


In [75]:

df.loc[df[df['bidder'] == " "].index,'bidder'] = "unknown"

In [76]:
data = df.groupby(['bidder']).agg({"prebid_win_count":"sum",
                                   "response_count":"sum"}).rename(columns={"prebid_win_count":"wins",
                                                                            "response_count":"total_bid"})
data['win_rate'] = (data['wins']/data['total_bid'])*100
data.sort_values(by='win_rate',ascending=False).head(2)

,wins,total_bid,win_rate
bidder,,,
undertone,52.0,104,50.000000
teads,323.0,700,46.142857


- Part 1: Describe how you would build a logistic regression model to predict the probability of a bid winning (win) based on the other columns. What features would you select and why?
- Part 2: Based on the following results from a logistic regression model  trained on the dataset, interpret the coefficients:
bid_amount: Coefficient = 0.8
time_to_bid: Coefficient = -0.4
ad_unit_A: Coefficient = 0.3 (reference category: ad_unit_B)

In [78]:
df.head(3)

,Unnamed: 0,date_hour,client,device_type,time_zone,os_name,browser_name,ad_unit,size,bidder,bid_range,media_type,request_count,response_count,prebid_win_count,win_count,sum_bid,sum_time_to_respond,median,min_bid,max_bid,avg_bid,sum_2nd_highest_bid,sum_prebid_winning_bid,sum_winning_bid,sum_nobid,sum_timeout
0,0,2021-04-11T01:00:00,asc126828,Desktop,America/Chicago,Linux,Chrome-85.04183133,div-gpt-ad-BT-D-Footers,728x90,onemobile,0.5-0.6,display,4,4,1.0,NaN,2.3408,6614.0,NaN,0.58520,0.58520,0.5852,0.5852,0.5852,NaN,NaN,NaN
1,1,2021-04-11T01:00:00,asc126828,Desktop,America/Chicago,Linux,Chrome-85.04183133,div-gpt-ad-BT-D-Footers,728x90,rhythmone,0.3-0.4,display,3,3,NaN,NaN,1.0323,5171.0,NaN,0.33945,0.34875,0.3441,NaN,NaN,NaN,NaN,NaN
2,2,2021-04-11T01:00:00,asc126828,Desktop,America/Chicago,Linux,Chrome-85.04183133,div-gpt-ad-BT-D-InLine-1,300x250,undertone,0.9-1,display,1,1,NaN,NaN,0.9600,1523.0,NaN,0.96000,0.96000,0.9600,NaN,NaN,NaN,NaN,NaN


- Part1

In [79]:
(df[['browser_name',
'size',
'bidder',
'bid_range',
'request_count',
'response_count',
'prebid_win_count',
'win_count',
'sum_time_to_respond',
'avg_bid']].isnull().sum()/df.shape[0])*100

browser_name            0.00
size                    0.00
bidder                  0.00
bid_range               0.00
request_count           0.00
response_count          0.00
prebid_win_count       76.66
win_count              96.28
sum_time_to_respond    24.20
avg_bid                24.20
dtype: float64

In [83]:
null_win = df[df['win_count'].isnull()].index
df.loc[null_win,'target'] = 0
target1_index = df[df['target'] !=0].index
df.loc[target1_index,'target'] = 1
df['target'].value_counts()

target
0.0    4814
1.0     186
Name: count, dtype: int64

In [ ]:
## check what all category we have in browser
df['browser_name'].value_counts()

browser_name
Chrome-89.04389114    2741
Chrome-89.043899       596
Chrome-86.04240198     223
Safari-14.03           201
Chrome-88.04324208     114
Chrome-61.0316379      112
Firefox-78             105
Chrome-83.04103106     103
Firefox-87             101
Chrome-88.04324192      86
Chrome-84.04147127      84
Chrome-89.04389116      61
Chrome-85.04183133      58
Chrome-87.04280142      50
Chrome-89.0438982       48
Safari-14.02            34
Chrome-84.04147136      28
Chrome-89.0438995       26
Chrome-88.0432419       24
Chrome-70.03538102      20
Chrome-59.0307186       16
Chrome-87.04280141      12
Mozilla-5               12
Firefox-86              12
Chrome-71.0357898       10
BlackBerry-1.1           9
Chrome-88.0432415        9
Chrome-91.04463          9
Chrome-92.04473          9
Chrome-88.04324182       8
Chrome-89.04389112       8
Safari-13.12             7
Firefox-85               7
Chrome-89.0438972        7
Chrome-87.0428088        6
Firefox-64               6
Chrome-53.02785

In [85]:
## for bid amount not all satatistics are importand, we can choose anyone
df[['sum_bid','median','min_bid','max_bid','avg_bid']].isnull().sum()

sum_bid    1210
median     3240
min_bid    1210
max_bid    1210
avg_bid    1210
dtype: int64

## part1
- target : 
    - if win_count_present : 1
    - if win_count is null : 0

- x features : 
- 'bidder' : some kind of bidder might after getting the chance can not choose to bid
- 'browser_name' : safari users are generally mac use, more chance to reach to people seeing ad and buying.
- 'size' :  bigger add size, less chance of selected for bidding , small size will also go for less chance of selction, moderate size might win
- 'bid_range' : lower bid range more chance of going for bidding
- 'response_count'/ 'request_count' : high chances of winning
- 'prebid_win_count' : more prebid win count, help bidder to take better decision wether to go for bid or not.
- 'sum_time_to_respond' : if response time will be too high there is less chance for bidder getting uninterested.
- 'avg_bid' : high bid price might not get responce.

## part2
- bid_amount: Coefficient = 0.8 : highly positive corelation: high bid amount has more chances of winning
- time_to_bid: Coefficient = -0.4 : negative coorelation: more time to bid less chance of winning
- ad_unit_A: Coefficient = 0.3 (reference category: ad_unit_B) : ad_unit_A is positively related with winning but
- it is less strongly related to wiining if we compare with ad_unit_B